# "Protein Pathfinders"
Capstone Project 2.0: Parkinson's disease

#### Notes:
- 4 separate and absolutely independent models for each of the 4 targets
- the observations that contain missing values in their respective target are excluded

#### Some insights from EDA
- KNN is found to work best on this data
- upd23b_clinical_state_on_medication does not make any difference in HP search
- however corr analysis shows that there is a linear correlation especially if you set nan to -1 (corr = +0.33 to +0.42) !
- correlation between visit_month and respective target:  +0.02 to +0.11
- Chi^2 and Cramer tests showed high / medium dependence (Cramer = 0.30 to 0.60) between columns 0 / -1 and the four target variables
- n_neighbors = 1 for all but target1 where n_neighbors is up to 5 >> try 1 to 6
- PCA: some targets use it some don't.  The lowest to try = 0.90
- No clear pattern on dropping columns 0,-1  (however drop them to predict exclusively on proteins/peptides)
- column_subsetter__threshold cannot be narrowed down
- no conclusive decision can be made as for log, imputer_strategy, scaler_special_columns, uniform/distance


## Pipeline Outline

### custom-made utility functions before the pipeline:

1. **get_data()**: load the spreadsheets, join, etc
2. **preprocess(data)**: imputes missing values in the target, can encode in X
(not necessary if the"alternative approach" is used)
3. alternative approach **separate(data)**: separate the targets and duplicate X, then drop rows with missing targets (individually for the four new datasets); for target 3 encode "upd23b_clinical_state_on_medication"; split each into train/test sets
4. **isolate_target(data)**: separate the target from the data (not necessary if the"alternative approach" is used)

### pipeline:
- **ColumnSubsetter**
    * excluding columns with nan's over a certain threshold <br>
    * dropping a particular column ('visit month' in our case) <br>

- **Encoder** for upd23b_clinical_state_on_medication
- **Imputer** : impute missing values
- **ElementwiseTransformer** (not necessary because we are using KNN)
- **Scaler**
- **FeatureSelector**:
    * model-based: DT-based, Lasso-based
    * univariate: selecting the features with a good correlation (check out the ready-made ones from sklearn)
    * dim reducing: PCA, KernalPCA
- **estimator** (multioutput estimator): any estimator - ensure integration for GridSearch - search for a good estimator 


### Domain knowledge: MDS-UPDRS

Part I: Nonmotor experiences of daily living: 13 items. Score range: 0–52, 
10 and below is mild, 22 and above is severe.

Part II: Motor experiences of daily living: 13 items. Score range: 0–52, 
12 and below is mild, 30 and above is severe.

Part III: Motor examination: 18 items. Score range: 0–132, 
32 and below is mild, 59 and above is severe.

Part IV: Motor complications: 6 items. Score range: 0–24, 
4 and below is mild, 13 and above is severe.


In [1]:
from time import perf_counter
import numpy as np, pandas as pd
from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import SelectPercentile, f_regression, SelectFromModel
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.dummy import DummyRegressor
from sklearn.decomposition import KernelPCA

from warnings import warn, filterwarnings
#filterwarnings('ignore', category=UserWarning)

np.set_printoptions(suppress=True, precision=3)

In [3]:
# custom-made utilities (utilities.py)
from utilities import smape_score, rmse_score, get_data, preprocess, isolate_target, separate
from utilities import Baseline, ColumnSubsetter, Encoder, Imputer, ElementwiseTransformer, Scaler, FeatureSelector
from utilities import ProteinPathfinders, AdHockExponentialDistribution
from utilities import print_report, print_overall_report, print_report_best_models

In [4]:
# random seed
SEED = 42

# Baseline model 
(excluding missing values, each of the 4 targets in isolation)

In [84]:
data = get_data(drop_columns=['patient_id', 'visit_month', 'upd23b_clinical_state_on_medication'],
               all_nans_targets='drop', full=False)
Xtrains, Xtests, ytrains, ytests = separate(data, stratify=True, test_split=0.25, random_state=SEED)

smapes = list()

for k in range(4):
    Xtrain, Xtest, ytrain, ytest = (obj[k] for obj in (Xtrains, Xtests, ytrains, ytests))
    base = Baseline('median').fit(Xtrain, pd.DataFrame(ytrain))
    SMAPE = base.score(Xtest, ytest)
    smapes.append(SMAPE)
    print(f"adjusted SMAPE for target {k}: {SMAPE:.0f}%")

print("overall SMAPE =", round(sum(smapes)/len(smapes)))    

adjusted SMAPE for target 0: 58%
adjusted SMAPE for target 1: 78%
adjusted SMAPE for target 2: 82%
adjusted SMAPE for target 3: 51%
overall SMAPE = 67


# Fetch data

In [86]:
drop_columns = ['patient_id',]
data, new_data = get_data(drop_columns, all_nans_targets='separate')
Xtrains, Xtests, ytrains, ytests = separate(data, return_arrays=False, test_split=0.25, random_state=SEED)

# Random HP Search

In [87]:
scorer = make_scorer(smape_score, greater_is_better=False)

In [88]:

steps = [
    ("encoder", Encoder()),    #encodes "upd23b_clinical_state_on_medication"
    ("column_subsetter", ColumnSubsetter()),
    ("imputer", Imputer()),     
    ("logger", ElementwiseTransformer()),
    ("scaler", Scaler()),                           
    #("feature_selector", FeatureSelector()),              
    ("estimator", KNeighborsRegressor(n_neighbors=None)),
        ]

pl = Pipeline(steps)

In [89]:
params = {
        'column_subsetter__threshold': uniform(0.00, 0.57),
        'imputer__strategy': ['mean', 'median'],
        'scaler__special_scaler': ["minmax", "standard"],       
        'estimator__n_neighbors': randint(1,6), 
        'estimator__weights': ['uniform', 'distance'],
            }

In [90]:
%%script echo skipping

n_iter = 10

results = []

for k in range(4):
    if k > 0:
        params['estimator__n_neighbors'] = randint(1,2)   # set to 1
    
    
    X_train, X_test, y_train, y_test = (ls[k] for ls in (Xtrains, Xtests, ytrains, ytests))

    start_time = perf_counter()
    result = RandomizedSearchCV(pl, params, scoring=scorer, cv=7, n_iter=n_iter).fit(X_train, y_train)
    results.append(result)
    print_report(results, X_train, y_train, X_test, y_test)
    print(f"time elapsed: {(perf_counter() - start_time)/60: .1f} minutes\n")


pp = ProteinPathfinders.from_sklearn_objects(results)
pp.print_metrics(Xtrains, ytrains, title="train")
pp.print_metrics(Xtests, ytests, title="test")
print_report_best_models(results)

skipping


## Grid HP Search

In [91]:
params = {
        'column_subsetter__threshold': np.linspace(0.01, 0.57, 57),
        'imputer__strategy': ['mean', 'median'],
        'scaler__special_scaler': ["minmax", "standard"],        
        'estimator__n_neighbors': range(1,6), 
        'estimator__weights': ['uniform', 'distance'],
            }

In [92]:

results = []

for k in range(4):
    if k > 0:
        params['estimator__n_neighbors'] = [1,2,3]
    
    
    X_train, X_test, y_train, y_test = (ls[k] for ls in (Xtrains, Xtests, ytrains, ytests))

    start_time = perf_counter()
    result = GridSearchCV(pl, params, scoring=scorer, cv=7).fit(X_train, y_train)
    results.append(result)
    print_report(results, X_train, y_train, X_test, y_test)
    print(f"time elapsed: {(perf_counter() - start_time)/60: .1f} minutes\n")


pp = ProteinPathfinders.from_sklearn_objects(results)
pp.print_metrics(Xtrains, ytrains, title="train")
pp.print_metrics(Xtests, ytests, title="test")
print_report_best_models(results)



 ***** TARGET 1 *****

2280 models were fitted
best estimator: KNeighborsRegressor
best parameters: {'column_subsetter__threshold': 0.18, 'estimator__n_neighbors': 2, 'estimator__weights': 'distance', 'imputer__strategy': 'median', 'scaler__special_scaler': 'minmax'}

CV SMAPE on train set: 52%, std = 2.09
SMAPE on train set: 0%
RMSE on train set:  0.0

SMAPE on test set: 48%
RMSE on test set: 4.79
________________________________________________________________________________
time elapsed:  25.9 minutes



 ***** TARGET 2 *****

1368 models were fitted
best estimator: KNeighborsRegressor
best parameters: {'column_subsetter__threshold': 0.019999999999999997, 'estimator__n_neighbors': 1, 'estimator__weights': 'uniform', 'imputer__strategy': 'mean', 'scaler__special_scaler': 'minmax'}

CV SMAPE on train set: 53%, std = 3.21
SMAPE on train set: 0%
RMSE on train set:  0.0

SMAPE on test set: 54%
RMSE on test set: 5.81
_____________________________________________________________________

# Create a single "Protein Pathfinders" model

In [93]:
model = ProteinPathfinders.from_sklearn_objects(results)
model

ProteinPathfinders(estimators=[Pipeline(steps=[('encoder', Encoder()),
                                               ('column_subsetter',
                                                ColumnSubsetter(drop_columns=[],
                                                                threshold=0.18)),
                                               ('imputer', Imputer()),
                                               ('logger',
                                                ElementwiseTransformer(skip_columns=[])),
                                               ('scaler',
                                                Scaler(special_columns=[],
                                                       special_scaler=MinMaxScaler())),
                                               ('estimator',
                                                KNeighborsRegressor(n_neighbors=2,
                                                                    weights='distance'))]),
                               P...
                                                KNeighborsRegressor(n_neighbors=1))]),
                               Pipeline(steps=[('encoder', Encoder()),
                                               ('column_subsetter',
                                                ColumnSubsetter(drop_columns=[],
                                                                threshold=0.09999999999999998)),
                                               ('imputer', Imputer()),
                                               ('logger',
                                                ElementwiseTransformer(skip_columns=[])),
                                               ('scaler',
                                                Scaler(special_columns=[],
                                                       special_scaler=MinMaxScaler())),
                                               ('estimator',
                                                KNeighborsRegressor(n_neighbors=1))])])

# Save the model

In [94]:
path = model.save()
path

'/Users/user/neuefische/Protein-Pathfinders/notebooks/protein_pathfinders_model_2023_04_26_15_47_04'

# Load the model

In [95]:
model = ProteinPathfinders.from_file(path)
model.print_metrics(Xtests, ytests, title='test')


OVERALL METRICS FOR TEST:
SMAPE (classic) = 34.4
SMAPE (adjusted) = 51.26
RMSE = 7.135
R² = 0.01958


# Load a model from a path 
(for example if you have run this notebook on Colab and saved the model file on local disc)

In [101]:
path = "protein_pathfinders_model_2023_04_26_15_47_04"   # your path here
model = ProteinPathfinders.from_file(path)
predictions = model.predict(Xtests)    # X_test could be new unseen data points you want to have predictions for
predictions   # jagged array because Xtests is a jagged array

array([array([10.491,  1.938,  1.983, 11.014,  4.544,  1.   ,  1.941, 12.406,
               8.89 ,  3.   ,  5.51 , 16.198, 13.935,  1.439,  3.405,  8.994,
               1.898, 10.91 ,  6.813,  4.817,  5.482,  5.702,  5.033,  2.097,
               6.669,  3.432,  4.679,  7.994,  6.151,  9.435,  8.651,  8.821,
              16.11 ,  8.105,  8.483,  8.44 ,  8.492,  6.502,  3.506,  7.101,
               9.832,  7.571,  7.845,  7.474,  7.338,  0.52 ,  1.683,  2.979,
               2.812,  7.683, 13.528,  1.501,  8.989,  2.026,  5.   ,  7.385,
              11.416,  3.992,  5.   ,  2.   , 10.017,  4.511, 10.965,  4.545,
              15.456,  4.009,  4.507,  1.009, 11.   ,  5.   , 10.524,  0.503,
               6.5  ,  5.596,  5.004,  2.513,  2.461,  1.991, 13.48 ,  6.051,
               9.948,  9.304,  4.014,  2.   ,  4.238,  3.391,  1.493,  8.546,
               9.991,  1.502, 13.463,  2.074,  3.186,  8.005,  4.483,  4.486,
               3.468,  7.   ,  6.   ,  6.221,  4.636,  5.562,  5

## Note:
If you don't have the file on your disk and the above cell threw an error, download this file from google drive and make sure it is saved under the correct name

In [97]:
google_drive_link = "https://drive.google.com/file/d/1C77uzw4KeyOwYLlc0FD92gHsVkoIyRYe/view?usp=sharing"

# Update selected estimators

for example if you ran this notebook on Colab and the (random) hyperparameter search returned a better result for a particular target.

In [98]:
%%script echo skipping

# example:
# a better estimator for target 4 from a run in Colab
target4 = {'column_subsetter__threshold': 0.26727244908528985, 'estimator__n_neighbors': 1, 
           'estimator__weights': 'distance', 'imputer__strategy': 'median', 
           'scaler__special_scaler': 'standard'}

model.update(4, target4)

skipping


# Refit the model
for example if the old data was expanded with some new observations
or you have updated estimators

In [99]:
model.fit(Xtrains, ytrains)
model.print_metrics(Xtests, ytests, title='test')


OVERALL METRICS FOR TEST:
SMAPE (classic) = 37.21
SMAPE (adjusted) = 54.75
RMSE = 6.654
R² = 0.09386


# Predict on new data

In [100]:
predictions = model.predict(new_data)
predictions.round().astype(int)

,updrs_1,updrs_2,updrs_3,updrs_4
2660_6,2,0,4,5
5027_6,2,0,0,0
5036_6,11,10,16,7
5178_6,2,0,3,0
7117_6,6,7,12,0
7151_6,10,15,26,7
11686_6,8,0,20,0
12636_6,4,6,24,0
12703_108,3,2,0,9
13968_6,4,0,4,0
